# keras 모델 안에 보려면 SVG - 상준 선임
tf==2.3.1


In [1]:
import tensorflow as tf
print(tf.__version__)

2023-01-12 15:08:25.597223: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda-10.0/lib64:/usr/local/cuda/extras/CUPTI/lib64
2023-01-12 15:08:25.597246: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


2.3.1


In [2]:
pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 2.4 MB/s eta 0:00:00m eta 0:00:010:00:02
  Using cached nvidia_cuda_runtime_cu11-11.7.99-py3-none-manylinux1_x86_64.whl (849 kB)
  Using cached nvidia_cuda_nvrtc_cu11-11.7.99-2-py3-none-manylinux1_x86_64.whl (21.0 MB)
  Using cached nvidia_cublas_cu11-11.10.3.66-py3-none-manylinux1_x86_64.whl (317.1 MB)
  Using cached nvidia_cudnn_cu11-8.5.0.96-2-py3-none-manylinux1_x86_64.whl (557.1 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import tensorflow as tf
from tensorflow import keras

model1 = tf.keras.models.load_model('./PRlab_student/')
model1.summary()

2023-01-12 15:10:05.405880: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-01-12 15:10:05.477371: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 15:10:05.477714: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1080 computeCapability: 6.1
coreClock: 1.835GHz coreCount: 20 deviceMemorySize: 7.93GiB deviceMemoryBandwidth: 298.32GiB/s
2023-01-12 15:10:05.477781: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-12 15:10:05.478095: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 1 with properties: 
pciBusID: 00

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 128, 128, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 128, 128, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 128, 128, 32) 0           bn_Conv1[0][0]                   
____________________________________________________________________________________________

# SMU Code


In [6]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from emotion import Emotion
import mediapipe as mp
import csv
import glob
import re
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
dirpath = "/media/sldev1/T7/DMS2/"
# filepath = ["영상1.mp4", "영상2.mp4"]
filepath = glob.glob(dirpath+'*.mp4')
filepath = sorted(filepath ,key=lambda s: int(re.findall(r'\d+', s)[3]))
filepath = filepath[4:] # 0~3.mp4들은 따로 코드작성할 것임.
emo = Emotion()
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

In [12]:
for i in tqdm(filepath):
    print(i)

    cap = cv2.VideoCapture(i)
    with mp_face_detection.FaceDetection(
        model_selection=0, min_detection_confidence=0.5) as face_detection:

        with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_Result_SL/'+str(i)[22:-4]+'.csv', 'w', newline = '') as file:
            writer = csv.writer(file)
            writer.writerow(["frame", "result"])

            while(cap.isOpened()):
                ret, frame = cap.read()
                rgbframe = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = face_detection.process(rgbframe)
                rgbframe.flags.writeable = True
                rgbframe = cv2.cvtColor(rgbframe, cv2.COLOR_RGB2BGR)

                if results.detections:
                    bbox = results.detections[0].location_data.relative_bounding_box
                    top = rgbframe.shape[0]*bbox.ymin
                    bottom = rgbframe.shape[0]*(bbox.xmin+bbox.height)
                    left = rgbframe.shape[1]*bbox.xmin
                    right = rgbframe.shape[1]*(bbox.xmin+bbox.width)

                    pre_emo, one, two = emo.process(frame[int(left)-10:int(right)+10, int(top)-10:int(bottom)+10])
                    # 0: neutral, 1: joy, 2: sadness, 3: surprise, 4: fear, 5: disgust, 6: anger, 7: contemp
                    cv2.putText(frame, f"Emotion : {pre_emo}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
                    cv2.putText(frame, f"Frame : {cap.get(1)}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 200), 2)
                    cv2.putText(frame, f"One : {one}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)
                    cv2.putText(frame, f"two : {two}", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 200, 0), 2)


                    cv2.rectangle(frame, (int(left), int(top)), (int(right), int(bottom)), (255, 255, 0), 3)

                cv2.imshow("frame", frame)

                if cv2.waitKey(1) == ord('q'):
                    break

                with open('/home/sldev1/github/TIL/Internship/DMS2_SMU/FER_Result_SL/'+str(i)[22:-4]+'.csv', 'a', newline = '') as file:
                    writer = csv.writer(file)
                    writer.writerow([cap.get(1), pre_emo])
                    file.close()


    cap.release()
    cv2.destroyAllWindows()

# 1258 frame에서 멈추는 중

  0%|                                                                                          | 0/46 [00:00<?, ?it/s]INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


/media/sldev1/T7/DMS2/05. 이민정_1209.mp4


  0%|                                                                                          | 0/46 [05:33<?, ?it/s]


KeyboardInterrupt: 

In [28]:
filepath = glob.glob(dirpath+'*.mp4')
# print(re.findall(r'\d+', filepath[1]))

filepath = sorted(filepath ,key=lambda s: int(re.findall(r'\d+', s)[3]))

for i in filepath:
    print(i)
    print(str(i)[22:-4])

/media/sldev1/T7/DMS2/01. 김형민_1201.mp4
01. 김형민_1201
/media/sldev1/T7/DMS2/02. 김두용_1205.mp4
02. 김두용_1205
/media/sldev1/T7/DMS2/03. 주용현_1206.mp4
03. 주용현_1206
/media/sldev1/T7/DMS2/04. 손하늘_1208.mp4
04. 손하늘_1208
/media/sldev1/T7/DMS2/05. 이민정_1209.mp4
05. 이민정_1209
/media/sldev1/T7/DMS2/06. 김세희_1213.mp4
06. 김세희_1213
/media/sldev1/T7/DMS2/07. 이선진_1213.mp4
07. 이선진_1213
/media/sldev1/T7/DMS2/08. 정승아_1214.mp4
08. 정승아_1214
/media/sldev1/T7/DMS2/09. 류채원_1214.mp4
09. 류채원_1214
/media/sldev1/T7/DMS2/10. 최서영_1214.mp4
10. 최서영_1214
/media/sldev1/T7/DMS2/11. 하현옥_1214.mp4
11. 하현옥_1214
/media/sldev1/T7/DMS2/12. 윤호영_1214.mp4
12. 윤호영_1214
/media/sldev1/T7/DMS2/13. 김설양_1215.mp4
13. 김설양_1215
/media/sldev1/T7/DMS2/14. 박세익_1215.mp4
14. 박세익_1215
/media/sldev1/T7/DMS2/15. 김주일_1215.mp4
15. 김주일_1215
/media/sldev1/T7/DMS2/16. 유현지_1215.mp4
16. 유현지_1215
/media/sldev1/T7/DMS2/17. 임우준_1216.mp4
17. 임우준_1216
/media/sldev1/T7/DMS2/18. 송영달_1216.mp4
18. 송영달_1216
/media/sldev1/T7/DMS2/19. 표성민_1216.mp4
19. 표성민_1216
/media/sldev